In [10]:
import duckdb

db_string = "/Users/patrick/dev/ucl/comp0158_mscproject/database/w2v.db"

In [2]:
# CREATE A TABLE
#con = duckdb.connect(database=':memory:')
con = duckdb.connect(database=db_string)  
duckdb.sql("\
    CREATE TABLE TEST (\
        ID VARCHAR,\
    )")
con.close()

In [5]:
# DESCRIBE
con = duckdb.connect(database=db_string)
res = duckdb.sql("DESCRIBE TEST")
print(res)
con.close()

┌─────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│ column_name │ column_type │  null   │   key   │ default │  extra  │
│   varchar   │   varchar   │ varchar │ varchar │ varchar │ varchar │
├─────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ ID          │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
└─────────────┴─────────────┴─────────┴─────────┴─────────┴─────────┘



### Load proteins

In [17]:
# load protein file into protein table
# 20 July 2024 - This took 12.9s to load uniprotkb-2759_78494531.dat (78M proteins)
con = duckdb.connect(database=db_string)           
con.execute("CREATE TABLE W2V_PROTEIN AS SELECT * FROM read_csv_auto('/Users/patrick/dev/ucl/comp0158_mscproject/data/protein/uniprotkb-2759_78494531.dat', columns={'id' :'VARCHAR', 'start': 'USMALLINT', 'end': 'USMALLINT'})")
con.close()

In [19]:
con = duckdb.connect(database=db_string)           
protein_count = con.execute("SELECT COUNT(*) FROM W2V_PROTEIN").fetchall()
print(protein_count)
con.close()

[(78494529,)]


In [10]:
con = duckdb.connect(database=db_string)      

# SELECT FROM LIST OF IDS - REALLY SLOW
#list = ['A0A010R6E0', 'A0A010RP22']
#entries = con.execute("SELECT * FROM PFAM_TOKEN WHERE column0 IN (SELECT UNNEST(?))", [list]).fetchall()

res = con.execute("SELECT * FROM PROTEIN WHERE ID = (?)", ['A0A010PZU8']).fetchall()

print(res)
con.close()

[('A0A010PZU8', 1, 1389)]


### LOAD PFAM TOKENS

In [15]:
# July 20 2024 - Took 1m 55s to load 296,017,815 entries
con = duckdb.connect(database=db_string)

con.execute("CREATE TABLE W2V_TOKEN AS SELECT * FROM read_csv_auto('/Users/patrick/dev/ucl/comp0158_mscproject/data/pfam/protein2ipr_pfam_20240715.dat', columns={'uniprot_id' :'VARCHAR', 'type' : 'VARCHAR', 'token' : 'VARCHAR', 'start': 'USMALLINT', 'end': 'USMALLINT'})")
con.close()

In [3]:
# July 20 2024 - Took 2min 10s to load 296,017,815 entries
con = duckdb.connect(database=db_string)

con.execute("CREATE TABLE W2V_PFAM_TOKEN AS SELECT * FROM read_csv_auto('/Users/patrick/dev/ucl/comp0158_mscproject/data/pfam/protein2ipr_pfam_20240715.dat', columns={'uniprot_id' :'VARCHAR', 'type' : 'VARCHAR', 'token' : 'VARCHAR', 'start': 'USMALLINT', 'end': 'USMALLINT'})")
con.close()

In [11]:
# with pfam only this shows 296,017,815 entries
# after loading disorder as well this shows 377,274,915 (81,257,100 disorder entries)
con = duckdb.connect(database=db_string)           
protein_count = con.execute("SELECT COUNT(*) FROM W2V_PFAM_TOKEN").fetchall()
print(protein_count)
con.close()

IOException: IO Error: Could not set lock on file "/Users/patrick/dev/ucl/comp0158_mscproject/database/w2v.db": Conflicting lock is held in /Users/patrick/miniconda3/envs/conda_ucl_base/bin/python3.12 (PID 1886) by user patrick. See also https://duckdb.org/docs/connect/concurrency

#### Load DISORDER tokens

In [22]:
# Load disorder entries
con = duckdb.connect(database=db_string) 
con.execute("INSERT INTO W2V_TOKEN SELECT * FROM read_csv_auto('/Users/patrick/dev/ucl/comp0158_mscproject/data/disorder/disordered_tokens_20240719.dat')")
con.close()

In [5]:
# Load disorder entries
# 20 July - Took 36.5s - 81,257,100 entries
con = duckdb.connect(database=db_string) 

con.execute("CREATE TABLE W2V_DISORDER_TOKEN AS SELECT * FROM read_csv_auto('/Users/patrick/dev/ucl/comp0158_mscproject/data/disorder/disordered_tokens_20240719.dat', columns={'uniprot_id' :'VARCHAR', 'type' : 'VARCHAR', 'token' : 'VARCHAR', 'start': 'USMALLINT', 'end': 'USMALLINT'})")
con.close()

In [6]:
# with pfam only this shows 296,017,815 entries
# after loading disorder as well this shows 377,274,915
con = duckdb.connect(database=db_string)           
protein_count = con.execute("SELECT COUNT(*) FROM W2V_DISORDER_TOKEN").fetchall()
print(protein_count)
con.close()

[(81257100,)]


In [ ]:
# with pfam only this shows 296,017,815 entries
# after loading disorder as well this shows 377,274,915
con = duckdb.connect(database=db_string)           
tokens = con.execute("SELECT * FROM W2V_DISORDER_TOKEN WHERE UNIPROT_ID=(?)", ['A0A010PZU8']).fetchall()
print(tokens)
con.close()

#### Apply indexes

In [ ]:
# THis breaks - runs out of memory
#con = duckdb.connect(database=db_string) 
#res = con.execute("CREATE INDEX W2V_TKN_IDX ON W2V_TOKEN(UNIPROT_ID)")
#con.close()

In [7]:
# took 1min 41s
con = duckdb.connect(database=db_string) 
res = con.execute("CREATE INDEX W2V_PF_TKN_IDX ON W2V_PFAM_TOKEN(UNIPROT_ID)")
con.close()

In [8]:
# took 14s
con = duckdb.connect(database=db_string) 
res = con.execute("CREATE INDEX W2V_DIS_TKN_IDX ON W2V_DISORDER_TOKEN(UNIPROT_ID)")
con.close()

### UTILITIES

#### Drop Table

In [16]:
con = duckdb.connect(database=db_string)           
con.execute("DROP TABLE PROTEIN")
con.close()

#### Unlock database

In [1]:
import duckdb
import os
db_path = "/Users/patrick/dev/ucl/comp0158_mscproject/database/proteins.db"

def is_locked():
    db_path = "/Users/patrick/dev/ucl/comp0158_mscproject/database/proteins.db"

    lock_file = f'{db_path}.lock'
    return os.path.exists(lock_file)

is_locked()

# this works from a command prompt
#fuser database/proteins.db

# then kill the id if there is one liset

False